In [1]:
!pip install pytorch-nlp
from torchnlp.datasets import imdb_dataset
from tqdm import tqdm
import re
from nltk import tokenize
import nltk
import pickle
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np
################################################################################
drive.mount('/content/drive', force_remount=True)
nltk.download('punkt')
train = imdb_dataset(train=True)
test = imdb_dataset(test=True)
################################################################################
def tokenizing(data, mode=0):
    if mode==1:
        text = ' '.join([s["text"] for s in data])
    else:
        text = data
    text = text.replace('<br /><br />',' ')
    text = text.replace('\x85','')
    text = text.lower()
    tokenized_sentence = re.compile('[.!?()]').split(text)
    tokenized_corpus = [None]*len(tokenized_sentence)
    if mode==1:
        for i in tqdm(range(len(tokenized_sentence))):
            tokenized_corpus[i] = tokenize.word_tokenize(tokenized_sentence[i])
    else:
        for i in range(len(tokenized_sentence)):
            tokenized_corpus[i] = tokenize.word_tokenize(tokenized_sentence[i])
    return tokenized_corpus
################################################################################
class MyDataset(Dataset):
  def __init__(self, x, y, z):
    self.data = x
    self.labels = y
    self.lengths = z

  def __len__(self):
    return len(self.labels)
  
  def __getitem__(self, index):
    return self.data[index], self.labels[index], self.lengths[index]
################################################################################
def custom_collate_fn(data):
    """
       data: is a list of tuples with (example, label, length)
             where 'example' is a tensor of arbitrary shape
             and label/length are scalars
    """
    _, labels, lengths = zip(*data)
    max_len = max(lengths)
    fix_size = data[0][0].size(1)
    features = torch.zeros((len(data), 1, fix_size, max_len))
    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)

    for i in range(len(data)):
      k, j = data[i][0][0].size(0), data[i][0][0].size(1)
      features[i][0] = torch.cat((data[i][0][0], torch.zeros((k, max_len - j))),1)

    return features.float(), labels.long(), lengths.long()
################################################################################
class Accuracy:
    """A class to keep track of the accuracy while training"""
    def __init__(self):
        self.correct = 0
        self.total = 0
        
    def reset(self):
        """Resets the internal state"""
        self.correct = 0
        self.total = 0
        
    def update(self, outputs, labels):
        """
        Updates the internal state to later compute the overall accuracy
        
        output: the output of the network for a batch
        labels: the target labels
        """
        _, predicted = torch.max(outputs.data, 1) # predicted now contains the predicted class index/label
        
        self.total += labels.size(0)
        self.correct += (predicted == labels).sum().item() # .item() gets the number, not the tensor
        #self.correct += ((outputs.data > 0.5) == labels).sum().item()

    def compute(self):
        return self.correct/self.total
################################################################################
def eval_accu():
  net.eval()
  eval_accuracy = Accuracy()
  eval_accuracy.reset()        
  with torch.no_grad():
      for eval_data in val_loader:
          # get the data points
          eval_inputs, eval_labels, eval_lengths = eval_data
          eval_inputs, eval_labels = eval_inputs.to(device), eval_labels.to(device)
          # forward the data through the network
          eval_outputs = net(eval_inputs.float())
          
          eval_accuracy.update(eval_outputs, eval_labels)
          
  print("\nTesting Accuracy: {:.2f}%".format(100 * eval_accuracy.compute()))
################################################################################
def vec(word):
  try:
    a = model.wv[word]
  except:
    a = np.zeros(300)
  return a

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
model = pickle.load(open('/content/drive/MyDrive/NLP/w2v_model_gensim.p','rb'))

In [2]:
model = pickle.load(open('/content/drive/MyDrive/NLP/glove-wiki-gigaword-300.p','rb'))

In [ ]:
c = 0
for a in tqdm(train):
  doc = tokenize.word_tokenize(a['text'])
  if len(doc)<301:
    c+=1
c

100%|██████████| 25000/25000 [00:52<00:00, 474.85it/s]


17259

In [3]:
random.shuffle(train)
vector_size = 300
x = torch.zeros((17259,1,300,300))
xindex = 0
y = []
z = []
for i in tqdm(range(25000)):
  doc = tokenize.word_tokenize(train[i]['text'])
  if len(doc)<301:
    x[xindex] = torch.cat((torch.reshape(torch.tensor(np.array([vec(word) for word in doc])), (1,vector_size, len(doc))),torch.zeros((1,300,300-len(doc)))),2)
    xindex += 1
    y.append(int(train[i]['sentiment']=='pos'))
    z.append(len(doc))
train_dataset = MyDataset(x,y,z)

100%|██████████| 25000/25000 [01:34<00:00, 264.95it/s]


In [4]:
random.shuffle(test)
x = []
y = []
z = []
for i in tqdm(range(2500)):
  doc = tokenize.word_tokenize(test[i]['text'])
  if len(doc)<301:
    doc = tokenize.word_tokenize(test[i]['text'])
    x.append(torch.cat((torch.reshape(torch.tensor(np.array([vec(word) for word in doc])), (1,vector_size, len(doc))),torch.zeros((1,300,300-len(doc)))),2))
    y.append(int(test[i]['sentiment']=='pos'))
    z.append(len(doc))
val_dataset = MyDataset(x,y,z)

100%|██████████| 2500/2500 [00:07<00:00, 318.66it/s]


In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=50, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=50, shuffle=True, num_workers=2)

In [21]:
net.eval()
i = 16
with torch.no_grad():
  a = F.softmax(net(train_dataset.data[i].to(device).float()),0)
print(a)
print(train_dataset.labels[i])

tensor([0.6954, 0.3046], device='cuda:0')
0


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (vector_size,2))
        self.conv2 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (vector_size,3))
        self.conv3 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (vector_size,4))
        self.conv4 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (vector_size,5))
        self.fc1 = nn.Linear(400, 2)
        self.dropout1 = nn.Dropout(0.30)
        self.dropout2 = nn.Dropout(0.30)
        self.dropout3 = nn.Dropout(0.30)
        self.dropout4 = nn.Dropout(0.30)
        self.dropout5 = nn.Dropout(0.30)
    def forward(self, x):
        x1 = F.relu(self.conv1(x))
        x1 = self.dropout1(F.max_pool2d(x1, (1,x1.size()[-1])))
        x2 = F.relu(self.conv2(x))
        x2 = self.dropout2(F.max_pool2d(x2, (1,x2.size()[-1])))
        x3 = F.relu(self.conv3(x))
        x3 = self.dropout3(F.max_pool2d(x3, (1,x3.size()[-1])))
        x4 = F.relu(self.conv4(x))
        x4 = self.dropout4(F.max_pool2d(x4, (1,x4.size()[-1])))
        x = torch.flatten(torch.cat((x1,x2,x3,x4),-3),start_dim=-3)
        x = self.fc1(self.dropout5(x))
        return x

net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()

In [7]:
#optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.5)
optimizer = optim.Adam(net.parameters(), lr=0.001)
net = net.float()
net.train()
live_accuracy = Accuracy()

for epoch in range(100):  # loop over the dataset multiple times
    print("\nStarting epoch {}".format(epoch+1))

    live_accuracy.reset()
    total = 0
    running_loss = 0.0

    # to make a beautiful progress bar
    loader = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, data in loader:
        # get the data points
        inputs, labels, lengths = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients (else, they are accumulated)
        optimizer.zero_grad()

        # forward the data through the network
        outputs = net(inputs.float())
        # calculate the loss given the output of the network and the target labels
        loss = criterion(outputs, labels)
        # calculate the gradients of the network w.r.t. its parameters
        loss.backward()
        # Let the optimiser take an optimization step using the calculated gradients
        optimizer.step()
        
        running_loss += loss
        total += outputs.size(0)

        live_accuracy.update(outputs, labels)
        loader.set_description("loss: {:.5f}|acc: {:.2f}%".format(running_loss/total,100 * live_accuracy.compute()))
    eval_accu()
    net.train()

print('Finished Training')


Starting epoch 1


loss: 0.01435|acc: 49.78%: 100%|██████████| 346/346 [00:19<00:00, 17.89it/s]



Testing Accuracy: 49.92%

Starting epoch 2


loss: 0.01391|acc: 55.31%: 100%|██████████| 346/346 [00:11<00:00, 29.73it/s]



Testing Accuracy: 49.92%

Starting epoch 3


...


Testing Accuracy: 53.07%

Starting epoch 6


loss: 0.01209|acc: 67.74%: 100%|██████████| 346/346 [00:11<00:00, 29.24it/s]



Testing Accuracy: 50.82%

Starting epoch 7


loss: 0.01139|acc: 69.85%:   4%|▍         | 13/346 [00:00<00:14, 22.73it/s]


KeyboardInterrupt: ignored

In [71]:
outputs

tensor([[ 0.0815, -0.5144],
        [ 0.2936, -0.0418],
        [ 0.0778, -0.0859],
        [ 0.1929, -0.5197],
        [ 0.1572,  0.0132],
        [ 0.1573, -0.6022],
        [ 0.3199,  0.1860],
        [ 0.5097, -0.8992],
        [ 0.2845, -0.4515],
        [-0.0651, -0.5852],
        [ 0.1037, -0.4975],
        [ 0.2961, -0.5361],
        [-0.1337, -0.4059],
        [ 0.2122, -0.0481],
        [ 0.4725,  0.0370],
        [ 0.3058, -0.3289],
        [ 0.7525,  0.0973],
        [ 0.2638, -0.2552],
        [ 0.7091, -0.1995],
        [-0.0984, -0.5223],
        [-0.1585, -0.0996],
        [ 0.0875, -0.4442],
        [-0.0515, -0.2641],
        [-0.0311,  0.0681],
        [ 0.4020, -0.7152],
        [ 0.2946, -0.3320],
        [ 0.3060, -0.3803],
        [ 0.3780, -0.6711],
        [-0.2936, -0.6758],
        [-0.1163, -0.4441],
        [ 0.0651,  0.1365],
        [ 0.1517, -0.2749],
        [ 0.2903, -0.3572],
        [ 0.6116, -0.2276],
        [ 0.1154,  0.0687],
        [ 0.2862, -0

In [ ]:
pickle.dump(net,open('/content/drive/MyDrive/NLP/imdb_sentiment.p','wb'))

In [ ]:
net = pickle.load(open('/content/drive/MyDrive/NLP/imdb_sentiment.p','rb'))

In [ ]:
net.eval()
accuracy = Accuracy()
accuracy.reset()
# Gradients are calculated on the forward pass for every iteration.
# As we do not need gradients now, we can disable the calculation.
with torch.no_grad():
    for data in tqdm(train_loader):
        # get the data points
        inputs, labels, lengths = data
        inputs, labels = inputs.to(device), labels.to(device)
        # forward the data through the network
        outputs = net(inputs.float())
        
        accuracy.update(outputs, labels)

print("Training Accuracy: {:.2f}%".format(100 * accuracy.compute()))

100%|██████████| 346/346 [00:09<00:00, 37.14it/s]

Training Accuracy: 85.30%


In [78]:
net.eval()
accuracy = Accuracy()
accuracy.reset()        
with torch.no_grad():
    for data in tqdm(val_loader):
        # get the data points
        inputs, labels, lengths = data
        inputs, labels = inputs.to(device), labels.to(device)
        # forward the data through the network
        outputs = net(inputs.float())
        
        accuracy.update(outputs, labels)
        
print("\nTesting Accuracy: {:.2f}%".format(100 * accuracy.compute()))

100%|██████████| 36/36 [00:01<00:00, 20.34it/s]


Testing Accuracy: 50.33%
